In [1]:
# connect to the device
from thorlabs_apt_device.devices.kdc101 import KDC101
from thorlabs_apt_device.protocol import *
'''
"rotationStage": {
            "managerName": "KDC101Manager",
            "managerProperties": {
                "port": "COM12",
                "units": "degrees",
                "posConvFac": 1919.641857862339,
                "velConvFac": 42941.66,
                "accConvFac": 14.66
            }
        }
https://www.thorlabs.com/Software/Motion%20Control/APT_Communications_Protocol.pdf

Sample stage S/N 27266343 Port 14 or 15
Camera stage S/N 27266327 Port 14 or 15
Actuator Z8235B  EncCnt per mm  34554.96  Velocity 772981.3692 (mm/s)  Acceleration 263.8443072 (mm/s2)
APT communication protocol page 39

Triggering the stage, information is in the APT communication protocol page 140

widget from abigail
in SetupStatusWidget:
"""Rotation stage"""
self.rotationStageHeader = QtWidgets.QLabel('Rotation stage')
self.rotationStageHeader.setFont(QtGui.QFont('Calibri', 14))
self.rotationStageHeader.setStyleSheet("font-weight: bold")

self.rotationStagePosLabel = QtWidgets.QLabel('Set position (deg) of rotation stage')
self.rotationStagePosEdit = guitools.BetterDoubleSpinBox()
self.rotationStagePosEdit.setDecimals(4)
self.rotationStagePosEdit.setMinimum(-360)
self.rotationStagePosEdit.setMaximum(360)
self.rotationStagePosEdit.setSingleStep(0.01)
self.jogStepSizeLabel = QtWidgets.QLabel('Set jog step size (deg)')
self.jogStepSizeEdit = guitools.BetterDoubleSpinBox()
self.jogStepSizeEdit.setDecimals(4)
self.jogStepSizeEdit.setMaximum(360)
self.jogStepSizeEdit.setSingleStep(0.01)
self.jogPositiveButton = guitools.BetterPushButton('>>')
self.jogNegativeButton = guitools.BetterPushButton('<<')
self.currentPosOfRotationStageLabel = QtWidgets.QLabel('Current position (deg) of rotation stage')
self.currentPosOfRotationStageDisp = QtWidgets.QLabel('')

'''
import time
manager_properties = {
    "port": "COM14",
    "units": "mm",
    "posConvFac": 34554.96,
    "velConvFac": 772981.3692,
    "accConvFac": 263.8443072
}

stage = KDC101(serial_port="COM14", home=False)
time.sleep(5)

#stage.set_jog_params(int(0.2*manager_properties['posConvFac']),
#                     int(2*manager_properties['accConvFac']), 
#                     int(1.5*manager_properties["velConvFac"]))
stage.set_velocity_params(int(20*manager_properties["accConvFac"]),
                          int(15*manager_properties["velConvFac"]))

In [3]:
stage.set_velocity_params(int(20*manager_properties["accConvFac"]),
                          int(15*manager_properties["velConvFac"]))

In [2]:
stage.status

{'position': 117969,
 'enc_count': 0,
 'velocity': 0,
 'forward_limit_switch': False,
 'reverse_limit_switch': False,
 'moving_forward': False,
 'moving_reverse': False,
 'jogging_forward': False,
 'jogging_reverse': False,
 'motor_connected': True,
 'homing': False,
 'homed': False,
 'tracking': False,
 'interlock': False,
 'settled': False,
 'motion_error': False,
 'motor_current_limit_reached': False,
 'channel_enabled': True,
 'msg': 'mot_get_dcstatusupdate',
 'msgid': 1169,
 'source': 80,
 'dest': 1,
 'chan_ident': 1,
 'forward_limit_soft': False,
 'reverse_limit_soft': True,
 'initializing': False,
 'instrument_error': False,
 'overtemp': False,
 'voltage_fault': False,
 'commutation_error': False,
 'digital_in_1': False,
 'digital_in_2': False,
 'digital_in_3': False,
 'digital_in_4': False,
 'encoder_fault': False,
 'overcurrent': False,
 'current_fault': False,
 'power_ok': True,
 'active': False,
 'error': False}

In [5]:
stage.set_jog_params()

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_atexit',
 '_close_port',
 '_error_callbacks',
 '_log',
 '_loop',
 '_port',
 '_process_message',
 '_run_eventloop',
 '_schedule_keepalives',
 '_schedule_reads',
 '_schedule_updates',
 '_thread',
 '_unpacker',
 '_write',
 'bays',
 'channels',
 'close',
 'controller',
 'genmoveparams',
 'genmoveparams_',
 'home',
 'homeparams',
 'homeparams_',
 'identify',
 'invert_direction_logic',
 'jogparams',
 'jogparams_',
 'keepalive_interval',
 'keepalive_message',
 'move_absolute',
 'move_jog',
 'move_relative',
 'move_velocity',
 'moverelparams_',
 'read_interval',
 'register_error_callback',
 'set_enabled',
 'set_home_params',
 '

In [3]:
stage.homeparams

{'home_dir': 0, 'limit_switch': 0, 'home_velocity': 0, 'offset_distance': 0}

In [2]:
# 2.
stage.set_home_params(772981*20, 10366)

In [2]:
# set trigger params
stage.set_triggerparams(
    trig1_mode=0,
    trig1_polarity=1,
    trig2_mode=0,
    trig2_polarity=1
    )

In [9]:
from typing import Optional, Sequence
import struct


def _pack(
    msgid: int,
    dest: int,
    source: int,
    *,
    param1: int = 0,
    param2: int = 0,
    data: Optional[bytes] = None
):
    if data is not None:
        assert param1 == param2 == 0
        return struct.pack("<HHBB", msgid, len(data), dest | 0x80, source) + data
    else:
        return struct.pack("<H2b2B", msgid, param1, param2, dest, source)


def mot_get_kcubetrigconfig(dest: int, source: int, chan_ident: int):
    return _pack(0x0525, dest, source, param1=chan_ident)

get_response = mot_get_kcubetrigconfig(stage.status['dest'],
                        stage.status['source'],
                        stage.status['chan_ident'])
print(get_response)

b'%\x05\x01\x00\x01P'


MGMSG_MOT_SET_KCUBETRIGIOCONFIG 0x0523
MGMSG_MOT_REQ_KCUBETRIGIOCONFIG 0x0524
MGMSG_MOT_GET_KCUBETRIGIOCONFIG 0x0525

MGMSG_MOT_SET_KCUBETRIGIOCONFIG 0x0523
MGMSG_MOT_REQ_KCUBETRIGIOCONFIG 0x0524
MGMSG_MOT_GET_KCUBETRIGIOCONFIG 0x0525

MGMSG_MOT_SET_KCUBEPOSTRIGPARAMS 0x0526
MGMSG_MOT_REQ_KCUBEPOSTRIGPARAMS 0x0527
MGMSG_MOT_GET_KCUBEPOSTRIGPARAMS 0x0528

In [10]:
print(stage.request_kcube_trigger_config(1))
print(stage.get_kcube_trigger_config())

None


TypeError: 'NoneType' object is not subscriptable

In [5]:
import os
import fnmatch
import re

def search_string_in_files(directory, search_string):
    matches = []
    pattern = re.compile(search_string)

    for root, dirnames, filenames in os.walk(directory):
        for filename in fnmatch.filter(filenames, '*'):
            filepath = os.path.join(root, filename)
            try:
                with open(filepath, 'r', encoding='utf-8', errors='ignore') as file:
                    for line_number, line in enumerate(file, 1):
                        if pattern.search(line):
                            matches.append({
                                'file': filepath,
                                'line_number': line_number,
                                'line': line.strip()
                            })
            except Exception as e:
                print(f"Could not read file {filepath}: {e}")

    return matches

# Example usage
directory_to_search = 'C:/Users/alm/git/Imswitch_descSPIM/'
#directory_to_search = 'C:/Users/alm/AppData/Roaming/Python/Python310/site-packages/thorlabs_apt_device/'
print(directory_to_search)
string_to_search = 'gain'
results = search_string_in_files(directory_to_search, string_to_search)

for match in results:
    print(f"Found in {match['file']} on line {match['line_number']}: {match['line']}")


C:/Users/alm/git/Imswitch_descSPIM/
Found in C:/Users/alm/git/Imswitch_descSPIM/descspim.json on line 11: "gain": 0,
Found in C:/Users/alm/git/Imswitch_descSPIM/LICENSE on line 191: modification of the work as a means of enforcing, against the work's
Found in C:/Users/alm/git/Imswitch_descSPIM/LICENSE on line 497: patent against the party.
Found in C:/Users/alm/git/Imswitch_descSPIM/README.md on line 34: - 594 modulation does not work again, there is no power modulation mode from python BUT in the software? email cobolt
Found in C:/Users/alm/git/Imswitch_descSPIM/README.md on line 40: - 2024-07-23 12:22:32 WARNING [ThorCamSciManager -> thorlabscam] Property gain does not exist, 2024-07-23 12:22:32 WARNING [ThorCamSciManager -> thorlabscam] Property blacklevel does not exist, remove blacklevel and gain from camera features
Found in C:/Users/alm/git/Imswitch_descSPIM/.git\hooks\pre-commit.sample on line 12: against=HEAD
Found in C:/Users/alm/git/Imswitch_descSPIM/.git\hooks\pre-commit.sa

In [16]:
# test calculation
posConvFac = 34554.96
posConvFac = 1/posConvFac * 1000
req_step = 1.3 
req_zstep_du = req_step * posConvFac * 1000

a = 200
b = 200
a2 = 800
b2 = 400

factor = abs(a - a2) / abs(b - b2)
factor = 3.2345
        
du_camera = [i for i in range(1, 100)]                          # device units camera
dist_camera = [i*posConvFac for i in du_camera]                 # distance camera in µm
dist_sample = [i*factor for i in dist_camera]                   # distance sample in µm
du_sample = [i/posConvFac for i in dist_sample]                 # device units sample
        
result = [(i, du) for i, du in enumerate(du_sample) if int(du) + 1 - du <= 0.1]
closest_du = min(result, key=lambda x: abs(x[1] - req_zstep_du))

sample_zstep = closest_du[1] * posConvFac
camera_zstep = dist_camera[closest_du[0]]


In [5]:

du_sample = [3.0000004999995, 6.000000999999, 9.0000014999985, 12.000001999998, 15.000002499997501, 18.000002999997, 21.0000034999965, 24.000003999996, 27.000004499995498, 30.000004999995003, 33.0000054999945, 36.000005999994, 39.0000064999935, 42.000006999993, 45.0000074999925, 48.000007999992, 51.0000084999915, 54.000008999990996, 57.000009499990504, 60.000009999990006, 63.00001049998949, 66.000010999989, 69.0000114999885, 72.000011999988, 75.00001249998752, 78.000012999987, 81.00001349998651, 84.000013999986, 87.0000144999855, 90.000014999985, 93.00001549998449, 96.000015999984, 99.0000164999835, 102.000016999983, 105.00001749998249, 108.00001799998199, 111.00001849998151, 114.00001899998101, 117.00001949998051]

for i, du in enumerate(du_sample):
    s = abs(int(du) - du)
    print(s, du, i)

4.999995000254387e-07 3.0000004999995 0
9.999990000508774e-07 6.000000999999 1
1.4999985005204053e-06 9.0000014999985 2
1.9999980001017548e-06 12.000001999998 3
2.499997501459461e-06 15.000002499997501 4
2.9999970010408106e-06 18.000002999997 5
3.4999964988458032e-06 21.0000034999965 6
3.9999960002035095e-06 24.000003999996 7
4.499995498008502e-06 27.000004499995498 8
4.999995002918922e-06 30.000004999995003 9
5.499994500723915e-06 33.0000054999945 10
5.999994002081621e-06 36.000005999994 11
6.4999935034393275e-06 39.0000064999935 12
6.9999929976916064e-06 42.000006999993 13
7.499992499049313e-06 45.0000074999925 14
7.999992000407019e-06 48.000007999992 15
8.499991501764725e-06 51.0000084999915 16
8.999990996017004e-06 54.000008999990996 17
9.499990504480138e-06 57.000009499990504 18
9.999990005837844e-06 60.000009999990006 19
1.0499989492984696e-05 63.00001049998949 20
1.099998900144783e-05 66.000010999989 21
1.1499988502805536e-05 69.0000114999885 22
1.1999988004163242e-05 72.0000119

In [6]:
result = [(i, du) for i, du in enumerate(du_sample) if abs(int(du) - du) <= 0.1]
print(result)

[(0, 3.0000004999995), (1, 6.000000999999), (2, 9.0000014999985), (3, 12.000001999998), (4, 15.000002499997501), (5, 18.000002999997), (6, 21.0000034999965), (7, 24.000003999996), (8, 27.000004499995498), (9, 30.000004999995003), (10, 33.0000054999945), (11, 36.000005999994), (12, 39.0000064999935), (13, 42.000006999993), (14, 45.0000074999925), (15, 48.000007999992), (16, 51.0000084999915), (17, 54.000008999990996), (18, 57.000009499990504), (19, 60.000009999990006), (20, 63.00001049998949), (21, 66.000010999989), (22, 69.0000114999885), (23, 72.000011999988), (24, 75.00001249998752), (25, 78.000012999987), (26, 81.00001349998651), (27, 84.000013999986), (28, 87.0000144999855), (29, 90.000014999985), (30, 93.00001549998449), (31, 96.000015999984), (32, 99.0000164999835), (33, 102.000016999983), (34, 105.00001749998249), (35, 108.00001799998199), (36, 111.00001849998151), (37, 114.00001899998101), (38, 117.00001949998051)]


In [2]:
round(0.80000)

1